# **10. Pipeline** **de** **procesamiento** **de** **texto** **e** **imagen** **de** **la** **receta**

Una vez desarrollada la funcionalidad de procesamiento de recetas con lenguaje natural y entrenado el modelo de procesamiento de imagenes, se define la pipeline de  asistencia virtual de cocina que las integra.

In [ ]:
import os
# Set the environment variable within the notebook
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

1. Carga del modelo de detección de ingredientes, '*best_new_6.pt*'.

In [ ]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO(r'C:\Users\Javier Ponsin\Desktop\Jorge\Ingredients-detection-YoloV8-1\best_new_6.pt')


2. Carga del input a utilizar en la pipeline del demostrador de la aplicación. Este input corresponde a una receta de tortilla de patatas procesada previamente mediante el módulo de lenguaje natural, a fin de extraer los pasos y los ingredientes asociados a cada uno.

In [ ]:
receta_tortilla = {'pasos': ['Pelamos las patatas y las cortamos en gajos finos.', 'Picamos la cebolla en trocitos.', 'Mezclamos las patatas con la cebolla en la sarten y lo freimos.', 'Una vez pochada las patatas y la cebolla, lo mezclamos con el huevo batido.', 'Por ultimo ponemos la mezcla en la sarten'], 'ingredientes': [['patata'], ['cebolla'], ['patata', 'cebolla'], ['patata', 'cebolla', 'huevo'], []]}

3. Dado que las etiquetas usadas para el modelo de lenguaje están en castellano y las del modelo YOLO en inglés, es necesario definir un diccionario que las relacione. Simplificamos el diccionario incluyendo solo aquellos ingredientes  comunes al procesador de texto y al de imagen.

Definimos la función ***traducir_ingredientes*** que añade una entrada a la receta con los ingredientes traducidos al inglés.

In [ ]:
traductor = {
    'patata': 'potato',
    'cebolla': 'onion',
    'huevo': 'egg',
    'leche': 'milk',
    'limon': 'lemon',
    'pan': 'bread',
    'pasta': 'pasta',
    'queso': 'cheese',
    'tomate': 'tomato',
    'zanahoria': 'carrot',
    'calabacin': 'zucchini'
}

In [ ]:
def traducir_ingredientes(receta, traductor):

    # Traduce cada sublista de ingredientes usando el diccionario
    ingredientes_en_ingles = []
    for lista in receta['ingredientes']:
        traducidos = [traductor.get(ing, ing) for ing in lista]  # Si no existe en traductor, deja el original
        ingredientes_en_ingles.append(traducidos)

    # Añade nueva clave con la traducción
    receta['ingredients'] = ingredientes_en_ingles
    return receta

In [ ]:
traducir_ingredientes(receta_tortilla, traductor)
print(receta_tortilla)

{'pasos': ['Pelamos las patatas y las cortamos en gajos finos.', 'Picamos la cebolla en trocitos.', 'Mezclamos las patatas con la cebolla en la sarten y lo freimos.', 'Una vez pochada las patatas y la cebolla, lo mezclamos con el huevo batido.', 'Por ultimo ponemos la mezcla en la sarten'], 'ingredientes': [['patata'], ['cebolla'], ['patata', 'cebolla'], ['patata', 'cebolla', 'huevo'], []], 'ingredients': [['potato'], ['onion'], ['potato', 'onion'], ['potato', 'onion', 'egg'], []]}


5. **Definición del algoritmo de asistencia de video y texto para recetas de cocina:**

El algoritmo propuesto se basa en sincronizar el avance de la receta mediante los ingredientes detectados en cada fotograma, intentando interpretar en cada instante en qué paso de la receta se encuentra el cocinero, mostrándole por pantalla al mismo el paso en el que se encuentra y el siguiente paso.

Para construir este algoritmo, se definen las siguientes funciones:

- *show_frame*: elige automáticamente el método adecuado para procesar la imagen según si la ejecución ocurre en Google Colab o en un entorno local.
- *_wrap_text*: divide un texto en varias líneas asegurando que cada una no exceda el ancho máximo. Se utiliza para definir el formato utilizado en *'show_pasos'*.
- *show_pasos*: genera y muestra una ventana en la que se visualizan, de forma formateada en texto, el paso actual y el siguiente de una receta.

Estas funciones se implementan en un bucle while, que es el núcleo del asistente en tiempo real. Este bucle  recorre el video frame a frame obteniendo mediante el modelo YOLO los ingredientes que aparecen en el mismo. Si en el frame detecta alguno perteneciente al siguiente paso de la receta, avanza el índice para actualizar el paso de la receta. La pantall de asistencia a la cocina se actualiza con los nuevos pasos actual y siguiente.

In [ ]:
import cv2
import numpy as np

def show_frame(img):

    try:
        from google.colab.patches import cv2_imshow
        from IPython.display import clear_output
        clear_output(wait=True)  # Clear output para evitar acumulación de imágenes
        cv2_imshow(img)
    except ImportError:
        cv2.imshow("YOLOv8 Tracking", img)

def _wrap_text(text, max_width, font, scale, thickness):
    """Parte el texto en líneas que quepan dentro de max_width (medido con cv2)."""
    words = text.split()
    lines, cur = [], ""
    for w in words:
        test = (cur + " " + w).strip()
        (wpx, _), _ = cv2.getTextSize(test, font, scale, thickness)
        if wpx <= max_width:
            cur = test
        else:
            if cur:
                lines.append(cur)
            cur = w
    if cur:
        lines.append(cur)
    return lines

def show_pasos(texto_paso_1, texto_paso_2, window_title="Pasos"):

    # Lienzo
    img_h, img_w = 420, 800
    img = np.ones((img_h, img_w, 3), dtype=np.uint8) * 255

    # Estilos
    font = cv2.FONT_HERSHEY_SIMPLEX
    scale_label = 0.85
    scale_text  = 0.8
    thick = 2
    x0, y0 = 16, 40
    line_gap = 8
    (_, base_h), _ = cv2.getTextSize("Ag", font, scale_text, thick)
    line_h = base_h + line_gap
    max_w = img_w - 2*x0

    y = y0

    # ---- Paso 1 ----
    cv2.putText(img, "Paso actual", (x0, y), font, scale_label, (0,0,160), thick, cv2.LINE_AA)
    y += int(line_h * 1.2)

    for line in _wrap_text(texto_paso_1, max_w, font, scale_text, thick):
        if y > img_h - 16:
            cv2.putText(img, "...", (x0, img_h-10), font, scale_text, (0,0,0), thick, cv2.LINE_AA)
            cv2.imshow(window_title, img)
            return
        cv2.putText(img, line, (x0, y), font, scale_text, (0,0,0), thick, cv2.LINE_AA)
        y += line_h

    y += int(line_h * 0.6)  # separación entre pasos

    # ---- Paso 2 ----
    cv2.putText(img, "Siguiente paso", (x0, y), font, scale_label, (0,0,160), thick, cv2.LINE_AA)
    y += int(line_h * 1.2)

    for line in _wrap_text(texto_paso_2, max_w, font, scale_text, thick):
        if y > img_h - 16:
            cv2.putText(img, "...", (x0, img_h-10), font, scale_text, (0,0,0), thick, cv2.LINE_AA)
            break
        cv2.putText(img, line, (x0, y), font, scale_text, (0,0,0), thick, cv2.LINE_AA)
        y += line_h

    cv2.imshow(window_title, img)

# Ruta del video
video = "C:/Users/Javier Ponsin/Desktop/Jorge/Ingredients-detection-YoloV8-1/tortilla9.mp4"

# Abrimos el video
cap = cv2.VideoCapture(video)

i = 0
paso = receta_tortilla['pasos'][i]
ingredients = receta_tortilla['ingredients'][i]
siguiente_paso = receta_tortilla['pasos'][i+1]

while cap.isOpened():
    success, frame = cap.read()

    if success:
        # Detección con YOLOv8
        results = model.track(frame, persist=True, verbose=False)
        result = results[0]

        # Extraemos ingredientes detectados (clases)
        if result.boxes.id is not None:

            for c in result.boxes.cls:
                ingredient = model.names[int(c)].lower()
                if ingredient in ingredients:
                    i = i
                elif ingredient in receta_tortilla['ingredients'][i + 1]:
                    i = i + 1
                    paso = receta_tortilla['pasos'][i]
                    siguiente_paso = receta_tortilla['pasos'][i+1]
                    ingredients = receta_tortilla['ingredients'][i]


        # Mostramos frame con anotaciones
        annotated_frame = result.plot()
        show_frame(annotated_frame)

        # Mostramos ventana con ingredientes
        show_pasos(paso,siguiente_paso)

        if cv2.waitKey(1) & 0xFF == ord('q'):  # Salir con 'q'
            break
    else:
        break

# Liberamos recursos
cap.release()
cv2.destroyAllWindows()